In [1]:
!pwd

/root/autodl-tmp/generativeModel/disguish_learning/LR_002


In [2]:
import sys
sys.path.insert(0, '/root/autodl-tmp/generativeModel/')

In [3]:

from disguish_learning.models.rm_pair_wise import rm_pair

In [4]:
import os

import transformers
from transformers import AutoTokenizer,AutoModel

import torch
from peft import (
    prepare_model_for_int8_training,
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
)
from datasets import load_dataset,load_from_disk
from peft import PeftModel


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
For effortless bug reporting copy-paste your error into this form: https://docs.google.com/forms/d/e/1FAIpQLScPB8emS3Thkp66nvqwmjTEgxp8Y9ufuWTzFyr9kJ5AoI47dQ/viewform?usp=sf_link
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /root/miniconda3/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/paths.py:27: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/paths.py:105: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/paths.py:27: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('Asia/Shanghai')}
  warn(
/root/miniconda3/lib/python3.8/site-packages/bitsandbytes/cuda_setup/paths.py:27: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64'), PosixPath('https')}
  

In [5]:
cache_dir = '/root/autodl-tmp/model/'
data_cache_dir = '/root/autodl-tmp/data/'
##############
# 模型部分 THUDM/glm-large-chinese 733m THUDM/glm-10b bigscience/bloom-7b1
base_model_name = 'bigscience/bloom-3b'
base_model =  AutoModel.from_pretrained(base_model_name,cache_dir=cache_dir)

model = rm_pair(base_model)

In [6]:
LORA_WEIGHTS=None#"/root/autodl-tmp/model/lora-alpaca"

lora_config = LoraConfig(
    r=8,#LORA_R,
    lora_alpha=16,#LORA_ALPHA,
    target_modules=["query_key_value"],#TARGET_MODULES,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
)
model = get_peft_model(model,lora_config)
# 最后一层还是要训练的
model.base_model.model.scorer.weight.requires_grad = True

In [21]:
for name,parameter in model.base_model.model.scorer.named_parameters():
    print(name,parameter.requires_grad)

weight True


In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name,cache_dir=cache_dir,trust_remote_code=True)


In [ ]:
model

In [10]:
data_cache_dir = '/root/autodl-tmp/data/'
dataset_name = "yitingxie-rlhf-reward-datasets"
dataset = load_from_disk(os.path.join(data_cache_dir,dataset_name))

In [ ]:
dataset['train'][0]['rejected']

In [ ]:
def build_tokenzie_func(tokenizer, pad_idx=0, max_length=1024, pad=True):
    def tokenize(example):
        """
        ignore label idx should always be -100
        """
        prompt_text = example['prompt']
        chosen_text = example['chosen']
        rejected_text = example['rejected']
        
        prompt_idxs = tokenizer(prompt_text)
        chosen_idxs = tokenizer(chosen_text)
        rejected_idxs = tokenizer(rejected_text)
        
        # positive
        positive = prompt_idxs['input_ids']  + chosen_idxs['input_ids'] 
        att_mask_pos = prompt_idxs['attention_mask']  + chosen_idxs['attention_mask'] 
        # negtive
        negtive = prompt_idxs['input_ids']  + rejected_idxs['input_ids'] 
        att_mask_neg = prompt_idxs['attention_mask']  + rejected_idxs['attention_mask'] 
        
        pos_actual_len = len(positive)
        neg_actual_len = len(negtive)
        # pad and truck
        positive = positive + [tokenizer.pad_token_id] * max(0,max_length - len(positive))
        att_mask_pos = att_mask_pos + [0] * max(0,max_length - len(att_mask_pos) )
        negtive = negtive + [tokenizer.pad_token_id] * max(0,max_length - len(negtive))
        att_mask_neg = att_mask_neg + [0] * max(0,max_length - len(att_mask_neg))
        positive = positive[:max_length]
        att_mask_pos = att_mask_pos[:max_length]
        negtive = negtive[:max_length]
        att_mask_neg = att_mask_neg[:max_length]
        
        
        example['pos_actual_len'] = pos_actual_len
        example['neg_actual_len'] = neg_actual_len
        
        example['positive'] = positive
        example['negtive'] = negtive
        
        #example['prompt_idxs'] = prompt_idxs
        example['att_mask_pos'] = att_mask_pos
        example['att_mask_neg'] = att_mask_neg
        #example['chosen_idxs'] = chosen_idxs
        #example['rejected_idxs'] = rejected_idxs
        
        
        
        return example

    return tokenize

In [ ]:
#print(train_data[0])

In [ ]:
tokenize_func = build_tokenzie_func(tokenizer,max_length=256)

In [ ]:
train_data = dataset['train'].map(tokenize_func)
test_data = dataset['test'].map(tokenize_func)

In [ ]:
type(train_data)

In [ ]:
print(train_data._data.num_rows)

In [ ]:
len(train_data)

In [ ]:
from dataclasses import dataclass
from transformers.data.data_collator import DataCollatorMixin

class data_collator_self(DataCollatorMixin):
    return_tensors: str = "pt"
    
    def batch_tensor_stack(self,features):
        assert len(features) > 0
        #print(features)
        batch = { 
            k : [v]
            for k,v in features[0].items()
        }
        #print(batch)
        
        for fea in features[1:]:
            #print("-"*100)
            #print(fea)
            for k,v in fea.items():
                batch[k].append(v)
        
        for k in batch:
            try:
                batch[k] = torch.tensor(batch[k],dtype=torch.long)
            except Exception as e:
                #print(k,e)
                batch[k] = batch[k]
        #print("-"*100)
        #print(batch)
        return batch
        
        
        
        
    def torch_call(self, features):
        import torch
        #print("--"*100)
        #print(features)
        
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature[label_name] for feature in features] if label_name in features[0].keys() else None

        no_labels_features = self.batch_tensor_stack(features)#[{k: v for k, v in feature.items() if k != label_name} for feature in features]


        
        batch = no_labels_features
        #print("-"*100)
        #print(batch)
        #print("-"*100)
        if labels is None:
            return batch

        batch[label_name] = torch.tensor(batch[label_name], dtype=torch.int64)
        #print(batch)
        return batch

In [ ]:
ddp = True if torch.cuda.device_count() > 1 else False
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=train_data,
    args=transformers.TrainingArguments(
        remove_unused_columns=False,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=128 // 1,
        warmup_steps=100,
        num_train_epochs=1,
        learning_rate=1e-5,
        fp16=True,
        logging_steps=20,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=200,
        save_steps=200,
        output_dir="lora-alpaca",
        save_total_limit=3,
        max_steps=10,
        load_best_model_at_end=True,
        ddp_find_unused_parameters=False if ddp else None,
    ),
    tokenizer=tokenizer,
    data_collator=data_collator_self()#transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.base_model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(self, old_state_dict())
).__get__(model, type(model))
if torch.__version__ >= "2":
    model = torch.compile(model)

##############
# 训练循环

trainer.train()


save_path = cache_dir + "/lora-alpaca/reward_model"
model.disable_adapter()
model.save_pretrained(save_path)
torch.save(model.get_base_model(),os.path.join(save_path,'model.bin'))

print("\n If there's a warning about missing keys above, please disregard :)")

In [ ]:
model.config